In [27]:
import torch
import torch.optim as optim
from torchsummary import summary
from tqdm import tqdm
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset,DataLoader
from torchvision.datasets import MNIST

In [28]:
MNIST_Transform=transforms.Compose([
    transforms.Pad((18,18),fill=0),
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,))
                                       ]
            )

In [29]:
mnist_train = MNIST(root='MNIST_DATASET/', # 다운로드 경로 지정
                          train=True, # True를 지정하면 훈련 데이터로 다운로드
                          transform=MNIST_Transform, # 텐서로 변환
                          download=True)

mnist_test = MNIST(root='MNIST_DATASET/', # 다운로드 경로 지정
                         train=False, # False를 지정하면 테스트 데이터로 다운로드
                         transform=MNIST_Transform, # 텐서로 변환
                         download=True)


train_set, val_set = torch.utils.data.random_split(mnist_train, [50000, 10000])


In [30]:
train_dl = torch.utils.data.DataLoader(dataset=train_set,
                                          batch_size=64,
                                          shuffle=True,
                                          drop_last=True)
valid_dl = torch.utils.data.DataLoader(dataset=val_set,
                                          batch_size=64,
                                          shuffle=False,
                                          drop_last=True)

In [31]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()

        self.layer1 = torch.nn.Sequential(
            nn.Conv2d(1, 32,3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer2 = torch.nn.Sequential(
            nn.Conv2d(32, 64, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer3 = torch.nn.Sequential(
            nn.Conv2d(64, 32, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
                
            
        self.layer4= torch.nn.Sequential(
            nn.Conv2d(32                    , 32, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
                    
        self.classifier = torch.nn.Sequential(
            nn.Linear(4 * 4 * 32, 625, bias=True),
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(625, 10, bias=True))

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out=self.layer4(out)
        out = out.view(out.size(0), -1)   # Flatten them for FC
        out = self.classifier(out)
        return out

In [32]:
model=CNN()

In [33]:
config_=dict()
config_["lr"]=0.01
config_["crit"]=nn.CrossEntropyLoss()
config_["optim"]=optim.Adam(model.parameters(),lr=config_["lr"])
config_["save_dir"]="models/"
config_["epochs"]=10
config_["train_log"]=[]
config_["valid_log"]=[]

config_["accuracy"]=True
config_["train_acc_log"]=[]
config_["valid_acc_log"]=[]
config_["log_interval"]=200
config_["device"]="cuda"


In [34]:
from train_skel import train

In [ ]:
config_,model=train(config_,train_dl,valid_dl,model)

Epochs : 1

Batch 1/781 Loss: 2.3056485652923584
Accuracy 0.0625

Batch 201/781 Loss: 2.3140652179718018
Accuracy 0.1109297263681592

Batch 401/781 Loss: 2.312519073486328
Accuracy 0.10995947630922694

Batch 601/781 Loss: 2.293179750442505
Accuracy 0.11090890183028286

Validation_Loss: 2.2983193397521973
Validation_Acc: 0.125
Epochs : 2

Batch 1/781 Loss: 2.298830032348633
Accuracy 0.078125

Batch 201/781 Loss: 2.302023410797119
Accuracy 0.10486629353233831


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(config_["train_log"])
plt.plot(np.arange(0,len(config_["train_log"]),len(config_["train_log"])/10),config_["valid_log"])

In [ ]:
import numpy as np
np.arange(0,len(config_["train_log"])+1,len(config_["train_log"])/10)